# WEB API

## Searching for all Train- and Busstations

In [1]:
from pandas import json_normalize
import requests
import json
import matplotlib.pyplot as plt

In [ ]:
# Overpass API URL
url = 'http://overpass-api.de/api/interpreter'

# Query
query = f"""
        [out:json];
        area["ISO3166-1"="CH"][admin_level=2];
        node ["public_transport"="stop_position"](area);
        out;"""

# Web API request
r = requests.get(url, params={'data': query})
data = r.json()['elements']

# Save data to file
with open('stations.json', 'w') as json_file:
    json.dump(data, json_file)

# Store data in data frame
df_stations = json_normalize(data)
df_stations.head(10)

## Plot Trainstations on map

In [3]:
"""
import folium
import pandas as pd

# Subset of railway Stations
locations = df_stations[["lat", "lon", "tags.operator", "tags.name", "tags.bus"]]
print(locations.head(5))

# Create map
map = folium.Map(location=[locations.lat.mean(), 
                           locations.lon.mean()], 
                 zoom_start=8, 
                 control_scale=True)

# Add maker symbols
for index, location_info in locations.iterrows():
    folium.Marker([location_info["lat"], 
                   location_info["lon"]], 
                  popup=location_info["tags.operator"]).add_to(map)

# Plot map
map
"""

'\nimport folium\nimport pandas as pd\n\n# Subset of railway Stations\nlocations = df_stations[["lat", "lon", "tags.operator", "tags.name", "tags.bus"]]\nprint(locations.head(5))\n\n# Create map\nmap = folium.Map(location=[locations.lat.mean(), \n                           locations.lon.mean()], \n                 zoom_start=8, \n                 control_scale=True)\n\n# Add maker symbols\nfor index, location_info in locations.iterrows():\n    folium.Marker([location_info["lat"], \n                   location_info["lon"]], \n                  popup=location_info["tags.operator"]).add_to(map)\n\n# Plot map\nmap\n'

In [4]:
"""
# Subset of railwaystations by SBB
locations = df_stations[["lat", "lon", "tags.operator", "tags.name", "tags.bus"]].loc[df_stations["tags.operator"] == 'SBB']
print(locations.head(5))

# Create map
map = folium.Map(location=[locations.lat.mean(), 
                           locations.lon.mean()], 
                 zoom_start=8, 
                 control_scale=True)

# Add maker symbols
for index, location_info in locations.iterrows():
    folium.Marker([location_info["lat"], 
                   location_info["lon"]], 
                  popup=location_info["tags.operator"]).add_to(map)

# Plot map
map
"""

'\n# Subset of railwaystations by SBB\nlocations = df_stations[["lat", "lon", "tags.operator", "tags.name", "tags.bus"]].loc[df_stations["tags.operator"] == \'SBB\']\nprint(locations.head(5))\n\n# Create map\nmap = folium.Map(location=[locations.lat.mean(), \n                           locations.lon.mean()], \n                 zoom_start=8, \n                 control_scale=True)\n\n# Add maker symbols\nfor index, location_info in locations.iterrows():\n    folium.Marker([location_info["lat"], \n                   location_info["lon"]], \n                  popup=location_info["tags.operator"]).add_to(map)\n\n# Plot map\nmap\n'

In [5]:
df_stations.to_csv('stations_data_geocoded.csv',
                      sep=",",
                      encoding='utf-8',
                      index=False)

In [6]:
import mysql.connector
from pandas.io import sql
from sqlalchemy import create_engine
import pymysql

conn = mysql.connector.connect(
  host="localhost",
  user="root",
  password="nh1hgE6L$1",
  database="projectDA_database")

print(conn)

# Create data base
cursor = conn.cursor()
cursor.execute("CREATE DATABASE IF NOT EXISTS projectDA_database")

# Show data bases in my sql
cursor.execute("SHOW DATABASES")

for x in cursor:
  print(x)

  # Create SQL-table Steuerfuss in the database
cursor.execute('''CREATE TABLE IF NOT EXISTS stations_table (id INT AUTO_INCREMENT PRIMARY KEY,
                                                               lat DECIMAL(12,8),
                                                               lon DECIMAL(12,8),
                                                               bus VARCHAR(50),
                                                               operator VARCHAR(50),
                                                               name VARCHAR(50))''')
# Confirm changes to the table
conn.commit()

('eatily',)
('exercise',)
('infections',)
('information_schema',)
('mysql',)
('performance_schema',)
('projectda_database',)
('projectthree',)
('projectx',)


In [7]:
df_for_database_stations = pd.read_csv('stations_data_geocoded.csv', sep=',', encoding='utf-8')[['lat', 'lon', 'tags.bus', 'tags.operator', 'tags.name']]
print(df_for_database_stations.shape)
df_for_database_stations.head(5)

NameError: name 'pd' is not defined

In [ ]:
db_data = 'mysql+mysqldb://' + 'root' + ':' + 'nh1hgE6L$1' + '@' + 'localhost' + ':3306/' \
       + 'projectda_database' + '?charset=utf8mb4'
engine = create_engine(db_data)

# Connect to the database
connection = pymysql.connect(host='localhost',
                         user='root',
                         password='nh1hgE6L$1',
                         db='projectda_database')   

# create cursor
cursor=connection.cursor()
# Execute the to_sql for writting DF into SQL
df_for_database_stations[['lat', 'lon', 'tags.bus', 'tags.operator', 'tags.name']].to_sql('stations_table', engine, if_exists='replace', index=False)    

# Execute query
sql = "SELECT * FROM `stations_table`"
cursor.execute(sql)

# Fetch all the records
result = cursor.fetchall()
for i in result:
    print(i)

engine.dispose()

In [ ]:
df_for_database_stations.lat.plot.hist(grid=True, 
                   bins=20, 
                   rwidth=0.9,
                   color='#607c8e')
plt.title('Gemeindesteuerfuss')
plt.xlabel('Gemeindesteuerfuss')
plt.ylabel('Frequency')
plt.grid(axis='y', alpha=0.75)


connection.close()